![clothing_classification](clothing_classification.png)


Fashion Forward is a new AI-based e-commerce clothing retailer.
They want to use image classification to automatically categorize new product listings, making it easier for customers to find what they're looking for. It will also assist in inventory management by quickly sorting items.

As a data scientist tasked with implementing a garment classifier, your primary objective is to develop a machine learning model capable of accurately categorizing images of clothing items into distinct garment types such as shirts, trousers, shoes, etc.

In [ ]:
#you need to install torchmetrics and torch vision
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchmetrics import Accuracy, Precision, Recall

In [55]:
# Load datasets
from torchvision import datasets
import torchvision.transforms as transforms

train_data = datasets.FashionMNIST(root='./data', train=True, download=True, transform=transforms.ToTensor())
test_data = datasets.FashionMNIST(root='./data', train=False, download=True, transform=transforms.ToTensor())

In [56]:
# Start coding here    
# Use as many cells as you need
images = train_data.data # This is a tensor of shape (60000, 28, 28)
print("images: ",images.shape)
labels = train_data.targets # This is a tensor of shape (60000,)
print("targets: ",labels.shape)

unique_labels = set(labels.numpy())
print(unique_labels) #{1, 2, 3, 4, 5, 6, 7, 8, 9}

image = images[0] #select the first image
print(image.shape)

images:  torch.Size([60000, 28, 28])
targets:  torch.Size([60000])
{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}
torch.Size([28, 28])


In [57]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchmetrics import Accuracy, Precision, Recall

# Ensure GPU is used if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#Define the CNN Model
class ProductTaggerCNN(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.feature_extractor = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),  # Grayscale images (1 channel)
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),

            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),

            nn.Flatten()
        )
        self.classifier = nn.Linear(64 * 7 * 7, num_classes)  # Adjusted to match the output of the feature extractor

    def forward(self, x):
        x = self.feature_extractor(x)
        x = self.classifier(x)
        return x

#Initialize Model, Loss, Optimizer
num_classes = 10  # Adjust based on dataset
model = ProductTaggerCNN(num_classes=num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

#DataLoader (Replace with actual dataset)
dataloader_train = DataLoader(train_data, batch_size=64, shuffle=True)
dataloader_test = DataLoader(test_data, batch_size=64, shuffle=False)

#Training Loop (1-2 epochs to keep runtime low)
for epoch in range(2):
    model.train()
    for images, labels in dataloader_train:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

#Evaluation and Storing Predictions
model.eval()
predictions = []
true_labels = []

# Metrics
metric_accuracy = Accuracy(task="multiclass", num_classes=num_classes).to(device)
# accuracy : Correct predictions / Total predictions = Measures overall correctness.
metric_precision = Precision(task="multiclass", num_classes=num_classes, average=None).to(device)
# precision : True Positives / (all predicted positive (both true and false))
metric_recall = Recall(task="multiclass", num_classes=num_classes, average=None).to(device)
# recall : True positives / (actual positives not just predicted) 

with torch.no_grad():
    for images, labels in dataloader_test:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)

        preds = torch.argmax(outputs, dim=1)
        predictions.extend(preds.cpu().tolist())  #  Store predictions as a list
        true_labels.extend(labels.cpu().tolist()) # labels is for the current batch, while true_labels holds the complete list of labels from the entire test set.

        metric_accuracy.update(preds, labels) #update keep adding data 
        metric_precision.update(preds, labels)
        metric_recall.update(preds, labels)

#Compute Final Metrics
accuracy = metric_accuracy.compute().item() # compute returns a tensor scalar and item convert it to python number
precision = metric_precision.compute().tolist()
recall = metric_recall.compute().tolist()

#Print Results
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")